In [ ]:
import pandas as pd
import sys
import io
from GeralModelY_ import *
from GeralModelYt_ import *
from openpyxl import load_workbook

pd.set_option('display.max_rows', None)

In [ ]:
instances = [
             'instancia1',
             'instancia2',
             'instancia3',
             'instancia4',
             'instancia5',
             'instancia6',
             'instancia7',
             'instancia8',
             'instancia9',
             'instancia10'
            ]

Qs = {
      'instancia1':{'P':561, 'Z':561},
      'instancia2':{'P':633, 'Z':633},
      'instancia3':{'P':419, 'Z':419},
      'instancia4':{'P':561, 'Z':561},
      'instancia5':{'P':417, 'Z':417},
      'instancia6':{'P':493, 'Z':493},
      'instancia7':{'P':491, 'Z':491},
      'instancia8':{'P':493, 'Z':493},
      'instancia9':{'P':419, 'Z':419},
      'instancia10':{'P':493, 'Z':493}
      }

classModels = [GeralModelY, GeralModelYt]

path_data = '/home/wilmer/Documentos/Codes/tesis/Instancias/reales/'

wb = load_workbook(path_data + 'resumo.xlsx')
hoja = wb.active

In [ ]:
for instance in instances:
    for classModel in classModels:

        tempModel = classModel(
            path_dem = path_data + instance + '/demanda.csv',
            path_preco = path_data + instance + '/preco.csv',
            path_rota1 = path_data + instance + '/rota.csv',
            Q = Qs[instance],
            perio = 0,
            nome = instance,
            abordagem = classModel.__name__,
            g_param = {
                't': 0, 
                'y': 5,
                'x': 11,
                'ng': 4,
                'attrs': ['Authorizations[Y]','Assignments[X]','Demanda','Preco'],
                }
        )

        metodos = [tempModel.independ_base_model,
                   tempModel.independ_fullfilment_model,
                   tempModel.independ_skiplagging_model,
                   tempModel.independ_complete_model,
                   tempModel.behavioral_base_model,
                   tempModel.behavioral_fullfilment_model,
                   tempModel.behavioral_skiplagging_model,
                   tempModel.behavioral_complete_model
                   ]
        
        for metodo in metodos:

            # Redigir a saída padrão para um arquivo
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout

            model, create_time, run_time, listNoUnimod =  metodo()

            # Voltar a saída padrão
            sys.stdout = old_stdout

            # Obter a saída da otimização
            output = new_stdout.getvalue()

            # Salvar a saída num arquivo de texto
            with open(path_data + instance + '/' + classModel.__name__ + '_' +  metodo.__name__ + '.txt', 'w') as f:
                f.write(output)

            # Salvar em excel um resumo dos resultados
            if model.status == GRB.OPTIMAL:
                hoja.append([instance, metodo.__name__, classModel.__name__, model.NumVars, create_time, run_time, model.NodeCount, model.IterCount, model.SolCount, model.MIPGap, model.ObjBound, model.ObjVal, model.ObjBound-model.ObjVal, len(listNoUnimod), model.NumConstrs, round((len(listNoUnimod)/model.NumConstrs)*100,2), listNoUnimod ])
            else:
                hoja.append([instance, metodo.__name__, classModel.__name__, model.NumVars, create_time, run_time, "-", "-", "-", "-", "-",'Infactível','-','-','-','-','-'])
            wb.save(path_data + 'resumo.xlsx')


            print("Termino a Instancia: " + instance + ", usando o modelo: " + metodo.__name__ + " da classe: " + classModel.__name__)

